# 1 задача

In [1]:
import numpy as np
import copy
from random import randint
from scipy.stats import uniform
from scipy.stats import norm
from scipy.stats import t
from scipy.stats import f
from scipy import integrate

N = 50

def random_vector():
    xi = uniform.rvs(scale=2, size=5) - 1
    return np.concatenate((xi, norm.rvs(loc=2 + np.matmul(xi, np.array([3, -2, 1, 1, -1]).T), scale=1.5, size=1)))


def make_sample(N):
    return np.array([random_vector() for _ in range(N)])


def linear_regression(X):
    N, dim = X.shape[0], X.shape[1] - 1
    Y = X.T[dim].T
    PSI = np.array([[1] + [X[j][i] for i in range(dim)] for j in range(N)])
    F = np.matmul(PSI.T, PSI)
    F1 = np.linalg.inv(F)
    BETA = np.matmul(np.matmul(F1, PSI.T), Y)
    e = Y - np.matmul(PSI, BETA)
    RSS = np.matmul(e.T, e)
    TSS = np.sum((Y - np.mean(Y)) ** 2)
    R = (1 - RSS / TSS) ** 0.5
    PVAL = []
    for i in range(len(BETA)):
        df = N - dim
        delta = BETA[i] * df ** 0.5 / (RSS * F1[i][i]) ** 0.5
        pval, err = integrate.quad(lambda x: t.pdf(x, df, loc=0, scale=1), np.abs(delta), np.inf)
        PVAL.append(2 * pval)
    return BETA, R, PVAL, e


def regression_info(X):
    beta, r, pval, err = linear_regression(X)
    print(f"""Уравнение лин. регрессии: 
    {round(beta[0], 2)} + {' + '.join(f'({round(beta[i + 1], 2)})*x{i + 1}' for i in range(beta.shape[0] - 1))}""")
    alpha = 0.05
    print("При этом коэффициент:")
    for i in range(len(pval)):
        print(f"\tb{i} {(pval[i] >= alpha) * 'не'} явл. значимым с p-val = {round(pval[i], 4)}")
    print()
    print(f"Коэффициент детерминации R = {r}")


def multicollinearity(X):
    Rs = []
    for k in range(X.shape[1] - 1):
        X_ = copy.copy(X.T)
        X_[X.shape[1] - 1] = X_[k]
        X_ = np.delete(X_, k, 0).T
        beta, r, pval, err = linear_regression(X_)
        Rs.append(r)
    return Rs


def collinearity_info(X):
    R_trust = 0.7
    R = multicollinearity(X)
    for k in range(len(R)):
        print(f"x{k + 1}:")
        print(f"\tR={round(R[k], 2)} < {R_trust}\n\t x{k + 1} лин. {(R[k] < R_trust) * 'не'}зависима от остальных")
        print()


def x0_trust(X, x0):
    Y = X.T[X.shape[1] - 1].T
    PSI = np.array([[1] + [X[j][i] for i in range(X.shape[1] - 1)] for j in range(X.shape[0])])
    F = np.matmul(PSI.T, PSI)
    F1 = np.linalg.inv(F)
    BETA = np.matmul(np.matmul(F1, PSI.T), Y)
    e = Y - np.matmul(PSI, BETA)
    RSS = np.matmul(e.T, e)
    PSI0x = np.array([1] + [x0[i] for i in range(X.shape[1] - 1)])
    A = 1 + np.matmul(np.matmul(PSI0x, F1), PSI0x.T)
    y0 = np.matmul(PSI0x, BETA.T)
    return y0, y0 + np.array([-2.0086, 2.0086]) * ((A * RSS) / (X.shape[0] - X.shape[1] - 1)) ** 0.5


def trust_info(X, x0):
    y0, dov = x0_trust(X, x0)
    print(f"Прогноз в x0 = {x0}:\n\ty0 = {y0}\nДоверительный интервал\n\t{dov}")


def errors(X):
    beta, r, pval, err = linear_regression(X)
    I = 0
    for i in range(len(err) - 1):
        for j in range(i + 1, len(err)):
            if err[i] > err[j]:
                I += 1
    delta = (I - len(err) * (len(err) - 1) / 4) / (len(err) ** 3 / 36) ** 0.5
    pval, err = integrate.quad(norm.pdf, np.abs(delta), np.inf)
    pval *= 2

    print(f"pval = {pval}\n"
          f"Гипотеза H0 {'не' * (pval <= 0.05)} является правдоподобной\n"
          f"Ошибки {'не' * (pval <= 0.05)} являются независимыми")


def outliers(X):
    beta, r, pval, err = linear_regression(X)
    e_a = np.abs(err)
    m = np.median(e_a)
    print("Выбросы выборки:")
    for i in range(len(err)):
        if not (-2 * m / 0.675 < err[i] < 2 * m / 0.675):
            print(f"\te{i} = {err[i]} != ({round(-2 * m / 0.675, 2)};{round(2 * m / 0.675, 2)})")


def cross_check(X):
    CV = []
    Y = X.T[X.shape[1] - 1].T
    for i in range(X.shape[0]):
        X_ = copy.copy(X)
        check_v = X_[i]
        X_ = np.delete(X_, i, 0)
        beta, r, pval, err = linear_regression(X_)
        CV.append((sum(np.concatenate((np.array([1]), check_v[0:X.shape[1] - 1])) * beta) - check_v[X.shape[1] - 1])**2)

    CVSS = sum(CV)
    TSS = np.sum((Y - np.mean(Y)) ** 2)
    RCV = ((TSS - CVSS) / TSS) ** 0.5

    return RCV


def cross_info(X):
    r = cross_check(X)
    print(f"Коэффициент детерминации Rcv = {r}")


def adequacy(X, n):
    N, dim = X.shape[0], X.shape[1] - 1
    Y = X.T[dim].T
    PSI = np.array([[1] + [X[j][i] for i in range(dim)] for j in range(N)])
    F = np.matmul(PSI.T, PSI)
    F1 = np.linalg.inv(F)
    BETA = np.matmul(np.matmul(F1, PSI.T), Y)
    e = Y - np.matmul(PSI, BETA)
    RSS = np.matmul(e.T, e)
    x0 = uniform.rvs(scale=2, size=dim) - 1
    
    print(f"Проверка в x0 = {x0}: {n} раз")
    eta = norm.rvs(loc=2 + np.matmul(x0, np.array([3, -2, 1, 1, -1]).T), scale=1.5, size=n)
    delta = RSS / ((N - dim) * np.sum((eta - np.mean(eta)) ** 2) / (n - 1))
    pval, err = integrate.quad(lambda x: f.pdf(x, N - dim, n - 1), delta, np.inf)
    
    print(f"pval = {pval}\n"
          f"Гипотеза H0 {'не' * (pval <= 0.05)} является правдоподобной\n"
          f"Регрессия {'не' * (pval <= 0.05)} является адекватной")


def del_maxp(X):
    beta, r, pval, err = linear_regression(X)
    minzn = np.max(pval)
    index = pval.index(minzn)
    print(f"Наим. значимость (pval = {round(minzn, 2)}) имеет b{index}")
    return np.delete(X.T, index, 0).T


def bootstrap(X, todel, n):
    N, dim = X.shape[0], X.shape[1] - 1
    beta, r, pval, err = linear_regression(X)
    r2 = r * r
    X_ = np.delete(X.T, todel, 0).T

    beta, r, pval, err = linear_regression(X_)
    r12 = r * r

    d = r2 - r12

    deltas = []
    for k in range(n):
        podX = np.array([X[randint(0, N - 1)] for _ in range(N)])
        podXdel = np.delete(podX.T, todel, 0).T
        beta, r, pval, err = linear_regression(podX)
        r2 = r * r
        beta, r, pval, err = linear_regression(podXdel)
        r12 = r * r
        delta = r2 - r12
        deltas.append(delta)

    deltas.sort()
    k = 0
    while deltas[k] < d:
        k += 1
    return (k - 1) / n


def bootstrap_info(X):
    regression_info(X)
    print()
    X_del = del_maxp(X)
    print()
    regression_info(X_del)
    print()
    beta, r, pval, err = linear_regression(X)
    minzn = np.max(pval)
    index = pval.index(minzn)
    pval = bootstrap(X, index, 1000)
    print(f"Удаление переменой {'не' * (pval <= 0.05)}имеет смысл(a) т.к. pval = {pval}")


X = make_sample(N)

## а

In [2]:
collinearity_info(X)

x1:
	R=0.41 < 0.7
	 x1 лин. независима от остальных

x2:
	R=0.27 < 0.7
	 x2 лин. независима от остальных

x3:
	R=0.21 < 0.7
	 x3 лин. независима от остальных

x4:
	R=0.38 < 0.7
	 x4 лин. независима от остальных

x5:
	R=0.22 < 0.7
	 x5 лин. независима от остальных



/var/folders/gv/c118vm9j4ndfhcj_8_jfnw1h0000gn/T/ipykernel_92071/3633361698.py:69: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  print(f"\tR={round(R[k], 2)} < {R_trust}\n\t x{k + 1} лин. {(R[k] < R_trust) * 'не'}зависима от остальных")


## b, c

In [3]:
regression_info(X)

Уравнение лин. регрессии: 
    1.6 + (2.88)*x1 + (-2.22)*x2 + (0.58)*x3 + (1.06)*x4 + (-1.42)*x5
При этом коэффициент:
	b0  явл. значимым с p-val = 0.0
	b1  явл. значимым с p-val = 0.0
	b2  явл. значимым с p-val = 0.0
	b3 не явл. значимым с p-val = 0.122
	b4  явл. значимым с p-val = 0.0113
	b5  явл. значимым с p-val = 0.0003

Коэффициент детерминации R = 0.8340294573745411


## d

In [5]:
trust_info(X, np.array([0 for _ in range(5)]))

Прогноз в x0 = [0 0 0 0 0]:
	y0 = 1.6004888280433622
Доверительный интервал
	[-1.25652755  4.45750521]


## e

In [7]:
errors(X)

pval = 0.363917791624866
Гипотеза H0  является правдоподобной
Ошибки  являются независимыми


## g

In [9]:
outliers(X)

Выбросы выборки:


## h

In [11]:
cross_info(X)

Коэффициент детерминации Rcv = 0.7751154042999068


## i

In [13]:
adequacy(X, 5)

Проверка в x0 = [-0.10306762  0.20011529  0.68606353 -0.54997521  0.11470595]: 5 раз
pval = 0.528636785407727
Гипотеза H0  является правдоподобной
Регрессия  является адекватной


## j

In [16]:
X_del = del_maxp(X)
print()
regression_info(X_del)

Наим. значимость (pval = 0.12) имеет b3

Уравнение лин. регрессии: 
    1.51 + (2.51)*x1 + (-2.26)*x2 + (0.54)*x3 + (-1.57)*x4
При этом коэффициент:
	b0  явл. значимым с p-val = 0.0
	b1  явл. значимым с p-val = 0.0
	b2  явл. значимым с p-val = 0.0
	b3 не явл. значимым с p-val = 0.1767
	b4  явл. значимым с p-val = 0.0001

Коэффициент детерминации R = 0.80519228094499


## k

In [18]:
bootstrap_info(X)

Уравнение лин. регрессии: 
    1.6 + (2.88)*x1 + (-2.22)*x2 + (0.58)*x3 + (1.06)*x4 + (-1.42)*x5
При этом коэффициент:
	b0  явл. значимым с p-val = 0.0
	b1  явл. значимым с p-val = 0.0
	b2  явл. значимым с p-val = 0.0
	b3 не явл. значимым с p-val = 0.122
	b4  явл. значимым с p-val = 0.0113
	b5  явл. значимым с p-val = 0.0003

Коэффициент детерминации R = 0.8340294573745411

Наим. значимость (pval = 0.12) имеет b3

Уравнение лин. регрессии: 
    1.51 + (2.51)*x1 + (-2.26)*x2 + (0.54)*x3 + (-1.57)*x4
При этом коэффициент:
	b0  явл. значимым с p-val = 0.0
	b1  явл. значимым с p-val = 0.0
	b2  явл. значимым с p-val = 0.0
	b3 не явл. значимым с p-val = 0.1767
	b4  явл. значимым с p-val = 0.0001

Коэффициент детерминации R = 0.80519228094499

Удаление переменой имеет смысл(a) т.к. pval = 0.546


# 

# 2 задача

In [25]:
import numpy as np
from scipy.stats import t
from scipy import integrate

X = np.array([[1,0,0,0,0,83],
              [1,0,0,0,0,85],
              [0,1,0,0,0,84],
              [0,1,0,0,0,85],
              [0,1,0,0,0,85],
              [0,1,0,0,0,86],
              [0,1,0,0,0,86],
              [0,1,0,0,0,87],
              [0,0,1,0,0,86],
              [0,0,1,0,0,87],
              [0,0,1,0,0,87],
              [0,0,1,0,0,87],
              [0,0,1,0,0,88],
              [0,0,1,0,0,88],
              [0,0,1,0,0,88],
              [0,0,1,0,0,88],
              [0,0,1,0,0,88],
              [0,0,1,0,0,89],
              [0,0,1,0,0,90],
              [0,0,0,1,0,89],
              [0,0,0,1,0,90],
              [0,0,0,1,0,90],
              [0,0,0,1,0,91],
              [0,0,0,0,1,90],
              [0,0,0,0,1,92]])


N, dim = X.shape[0], X.shape[1] - 1

PSI = X.T[0:dim].T
Y = X.T[dim].T

F = np.matmul(PSI.T,PSI)
F1 = np.linalg.inv(F)
beta = np.matmul(np.matmul(F1, PSI.T), Y)

e = Y - np.matmul(PSI, beta)
RSS = np.matmul(e.T,e)
TSS = np.sum((Y-np.mean(Y))**2)
r = (1 - RSS / TSS) ** 0.5

PVAL = []
for i in range(len(beta)):
    df = N-dim
    delta = beta[i] * (df) ** 0.5 / (RSS * F1[i][i]) ** 0.5

    def func(x):
        return t.pdf(x, df, loc=0, scale=1)
    pval, err = integrate.quad(func, np.abs(delta), np.inf)
    PVAL.append(2*pval)

pvald = []
inds = []
for i in range(len(beta) - 1):
    for j in range(i+1, len(beta)):
        inds.append((i, j))
        df = N-dim
        delta = (beta[i] - beta[j]) * (df) ** 0.5 / (RSS * (F1[i][i] + F1[j][j])) ** 0.5

        def func(x):
            return t.pdf(x, df, loc=0, scale=1)
        pval, err = integrate.quad(func, np.abs(delta), np.inf)
        pvald.append(2 * pval)

print(f"""Уравнение линейной регрессии: 
{round(beta[0], 2)} + {' + '.join(f'({round(beta[i + 1], 2)})*x{i + 1}' for i in range(beta.shape[0] - 1))}""")

print("При этом коэффициент:")
for i in range(len(PVAL)):
    print(f"\tb{i} {(PVAL[i] >= 0.05) * 'не'} явл. значимым с p-val = {round(PVAL[i],4)}")
print()

print(f"Коэфф. детерм. r = {r}\n")

print("Попарно коэфф.:")
for i in range(len(pvald)):
    print(f"\tb{inds[i][0]} {(pvald[i]<0.05)*'!'}= b{inds[i][1]} с p-val = {round(pvald[i],4)}")
print()


Уравнение линейной регрессии: 
84.0 + (85.5)*x1 + (87.82)*x2 + (90.0)*x3 + (91.0)*x4
При этом коэффициент:
	b0  явл. значимым с p-val = 0.0
	b1  явл. значимым с p-val = 0.0
	b2  явл. значимым с p-val = 0.0
	b3  явл. значимым с p-val = 0.0
	b4  явл. значимым с p-val = 0.0

Коэфф. детерм. r = 0.90033663737852

Попарно коэфф.:
	b0 = b1 с p-val = 0.1031
	b0 != b2 с p-val = 0.0002
	b0 != b3 с p-val = 0.0
	b0 != b4 с p-val = 0.0
	b1 != b2 с p-val = 0.0004
	b1 != b3 с p-val = 0.0
	b1 != b4 с p-val = 0.0
	b2 != b3 с p-val = 0.0024
	b2 != b4 с p-val = 0.001
	b3 = b4 с p-val = 0.2958

